In [1]:
from dgl.data import citation_graph,CoraDataset,TUDataset,GINDataset, ICEWS18
import torch
from dgl import DGLGraph
import numpy as np
from datasets.prepocessing import one_class_processing

In [2]:
#data =citation_graph.load_cora()
#data =citation_graph.load_citeseer()
data =TUDataset('PROTEINS_full')
#data =ICEWS18('train')

In [3]:
from dgl.data import tu
dataset = tu.LegacyTUDataset(name='PROTEINS_full')
train_size = int(0.6 * len(dataset))
test_size = int(0.2 * len(dataset))
val_size = int(len(dataset) - train_size - test_size)

dataset_train, dataset_val, dataset_test = torch.utils.data.random_split(
    dataset, (train_size, val_size, test_size))

In [4]:
input_dim, label_dim, max_num_node = dataset.statistics()

In [18]:
print(len(dataset_train))
print(len(dataset_val))
print(len(dataset_test))

667
224
222


In [21]:
dataset[0][0]

DGLGraph(num_nodes=42, num_edges=162,
         ndata_schemes={'feat': Scheme(shape=(29,), dtype=torch.float64)}
         edata_schemes={})

In [28]:
label0=data[0][0].ndata['node_labels']
label1=data[1][0].ndata['node_labels']
label2=data[2][0].ndata['node_labels']

In [31]:
label1

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])

In [29]:
print(len(label0))
print(len(label1))
print(len(label2))

42
27
10


In [5]:
data[1][0].ndata['node_labels'].size()

torch.Size([27, 1])

In [7]:
labels,train_mask,val_mask,test_mask=one_class_processing(data[0][0],4)

AttributeError: 'DGLSubGraph' object has no attribute 'labels'

In [ ]:
from collections import Counter
Counter(data.labels)

In [ ]:
normal_class=2
labels,train_mask,val_mask,test_mask=one_class_processing(data.labels,normal_class)

In [ ]:
labels[train_mask.astype(int)].shape

In [ ]:
print('train',np.sum(labels[train_mask.astype(int)==1]))
print('val',np.sum(labels[val_mask.astype(int)==1]))
print('test',np.sum(labels[test_mask.astype(int)==1]))

In [ ]:
print('train',np.sum(train_mask))
print('val',np.sum(val_mask))
print('test',np.sum(test_mask))

In [ ]:
train_mask.shape